### 1. Install pytest and ipytest (not using don't need install)

In [259]:
#!pip install pytest ipytest

### 2. Import

In [258]:
import re
#import asyncio
#import concurrent.futures
#import pytest
#import ipytest

### 3. Car object

In [175]:
class Car:
    def __init__(self,x,y,dir,commands):
        self.x = x
        self.y = y
        self.dir = dir
        self.commands = commands
        
    def __str__(self):
        if self.dir == 0:
            d = "N"
        elif self.dir == 1:
            d = "E"
        elif self.dir == 2:
            d = "S"
        else:
            d = "W"
        return "({},{}) {}, {}".format(self.x, self.y,d,self.commands)
        
    def PrintWithoutCommands(self):
        if self.dir == 0:
            d = "N"
        elif self.dir == 1:
            d = "E"
        elif self.dir == 2:
            d = "S"
        else:
            d = "W"
        return "({},{}) {}".format(self.x, self.y,d)

    def GetX(self):
        return self.x
        
    def GetY(self):
        return self.y

    def GetDir(self):
        return self.dir
        
    def RotateLeft(self):
        self.dir -= 1
        if self.dir < 0:
            self.dir = 3
            
    def RotateRight(self):
        self.dir += 1
        if self.dir > 3:
            self.dir = 0

    def SetPosition(self, x , y):
        self.x = x
        self.y = y

    def GetCommandsLen(self):
        return len(self.commands)
            
    def GetCommand(self, index):
        if index > len(self.commands) - 1 :
            return "";
        else:
            return self.commands[index];


### 4. Session object

In [176]:
class Session:
    def __init__(self):
        self.field_x = 0
        self.field_y = 0
        self.field = [ [ "" for i in range(self.field_x) ] for j in range(self.field_y) ]
        # car dictionary list
        #{ "name":car object}
        self.car_dict_list = {}
        # collided field dictionary list
        #{ "(x,y)":"A,B,C"} 
        self.collided_info_dict_list = {}

    def ConstructEmptyField(self):
        self.field = [ [ "" for i in range(self.field_x) ] for j in range(self.field_y) ]
        
    def FieldConstruct(self):
        while True:
            print("Please enter the width and height of the simulation field in x y format:")
            x = 0
            y = 0
            try:
                x,y = map(int,input().split())
            except ValueError:
                print("Invalid input.")
                continue
            if x > 0 and y > 0:
                self.field_x = x
                self.field_y = y
                self.ConstructEmptyField()
                print("you have created a field of",self.field_x,"x",self.field_y,".")
                break
            print("The width and height must be greater than 0.")

    # use for testing without prompt
    def test_FieldConstruct(self,x,y):
        self.field_x = x
        self.field_y = y
        self.ConstructEmptyField()

    def AddCar(self):
        # asking user the name of the car, which is also key in the dictionary
        while True:
            print("Please enter the name of the car:")
            name = input()
            pattern = "^[A-Za-z0-9_-]*$"
            if re.match(pattern, name) == False:
                print("name can only contains letters, numbers, underscores and dashes.")
                continue
            if any(name in d for d in self.car_dict_list):
                print(name, "existed in the list of car.")
            else:
                break
        # asking user the init position and the direction
        while True:
            print("Please enter initial position of car",name,"in x y Direction format:")
            input_list = input().split()
            if len(input_list) != 3:
                print("Invaild number of input.")
                continue
            try:
                x = int(input_list[0])
                y = int(input_list[1])
            except ValueError:
                print("x and y must be an integer.") 
                continue
            if x < 0 or x > self.field_x - 1 :
                print(x," is out of bound.")
                continue
            if y < 0 or x > self.field_y - 1 :
                print(y," is out of bound.")
                continue
            if self.field[x][y] != "":
                print("Field[",x,"][",y,"] already occupied with car:", self.field[x][y],".")
                continue
            if input_list[2] == "N":
                d = 0 
                break
            elif input_list[2] == "E":
                d = 1
                break
            elif input_list[2] == "S" :
                d = 2
                break
            elif input_list[2] == "W" :
                d = 3
                break
            print("Invalid letter for direction input.")
        # Asking the user for the commands 
        while True:
            print("Please enter the commands for car",name,":")
            commands_list = input()
            if any(c not in 'FLR' for c in commands_list):
                print("Commands contain invalid characters")
                continue
            break
        # Add new create car into a list of dictionary and print the list
        self.car_dict_list[name]= Car(x,y,d,commands_list)
        self.field[x][y] = name
        self.DisplayCarDictList()

    def DisplayCarDictList(self):
        print("Your current list of cars are:")
        for key, value in self.car_dict_list.items():  
            print(key,",", value)

    #use for testing without prompt
    def test_AddCar(self,name,x,y,input_d,commands_list):
        d = 0
        if input_d == "N":
            d = 0 
        elif input_d == "E":
            d = 1
        elif input_d == "S" :
            d = 2
        elif input_d == "W" :
            d = 3
        else:
            print("Invalid direction input")
            return
        self.car_dict_list[name]= Car(x,y,d,commands_list)

    def CheckCollision(self,x,y,key):
        if self.field[x][y]  == "":
            self.field[x][y] = key
            return False
        
        # "(x,y)" as collided_info_dict_list key
        collided_key = "("+ str(x) + "," + str(y) + ")" 
        if any(collided_key in d for d in self.collided_info_dict_list):
            # append with the new car into the value
            self.collided_info_dict_list[collided_key]= self.collided_info_dict_list[collided_key]+","+key
        else:
            # Add both cars into the value
            self.collided_info_dict_list[collided_key] = self.field[x][y] + "," + key
        return True

    def ExecuteCommand(self,key,value,step):
        # command possible give empty string "", mean the command already finish
        command = value.GetCommand(step)
        x = value.GetX()
        y = value.GetY()
        if command == "L":
            value.RotateLeft()
        elif command == "R":
            value.RotateRight()
        elif command == "F":
            d = value.GetDir()
            if d == 0:
                y += 1
            elif d == 1:
                x += 1
            elif d == 2:
                y -= 1
            else:
                x -= 1
            # not move as out-of-field boundary
            if x < 0 or y < 0 or x > self.field_x - 1 or y > self.field_y - 1:
                x = value.GetX()
                y = value.GetY()
            value.SetPosition(x,y)
            
        return self.CheckCollision(x,y,key)
        
       
                
    def PrintCollidedMessage(self,step):
        for key, value in self.collided_info_dict_list.items():
            cars = value.split(",")
            for i in range(len(cars)):
                exculded_ownself = ""
                if i == 0:
                    exculded_ownself = value.replace(cars[i] + ",","")
                else:
                    exculded_ownself = value.replace("," + cars[i],"")
                print("-", cars[i],", collides with",exculded_ownself, "at",key,"at step",str(step))
              
                            
    def RunSimulation(self):
        longest_commands = 0
        collided = False
        step = 0
        # The idea is every step needs to empty the field and then mark the position again
        self.ConstructEmptyField()
        # First loop check which has the longest commands
        # This possibly does it with concurrent
        for key, value in self.car_dict_list.items():
            longest_commands = max(longest_commands,value.GetCommandsLen()-1)  
            collided = self.ExecuteCommand(key,value,step) or collided
       
        while collided == False and step < longest_commands:
            step = step + 1
            self.ConstructEmptyField()
            # This possibly does it with concurrent
            for key, value in self.car_dict_list.items():
                collided = self.ExecuteCommand(key,value,step) or collided
                    
        #last checking is it collided or successfully executed all with not collided     
        print("After Simulation the result is:")
        if collided == True:
           self.PrintCollidedMessage(step + 1)
        else:
            for key, value in self.car_dict_list.items():
                print("-",key,",",value.PrintWithoutCommands())
            

### 5. Test case

In [265]:
# test_AddCar(string: name, int:x, int:y, string: direction(only accept "N", "E","S","W"), string: command)
# There is no type of valid check, except direction, make sure to put correct type of input
# Example
# test_sim.test_AddCar("A",0,0, "N", "LL")
#ipytest.autoconfig()

def test_RotateLeft():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",0,0,"N","LL")
    test_sim.test_AddCar("B",1,1,"W","LLLL")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()

def test_RotateRight():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",0,0,"N","RRR")
    test_sim.test_AddCar("B",1,1,"E","RRRR")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()
    
def test_OutOfBound():
    test_sim = Session()
    test_sim.test_FieldConstruct(1,1)
    test_sim.test_AddCar("A",0,0,"N","F")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()
   
def test_NoCommand():
    test_sim = Session()
    test_sim.test_FieldConstruct(1,1)
    test_sim.test_AddCar("A",0,0,"N","")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()
    # can't solve caofd not defined problem
    #out, err = capfd.readouterr()
    #expected_output = (
    #    "Your current list of cars are:\n"
    #    "A , (0,0) N, F\n"
    #   "After Simulation the result is:\n"
    #   "- A , (0,0) N"
    #)
    #assert out.strip() == expected_output.strip()


def test_NoCollided():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",0,0,"N","FFF")
    test_sim.test_AddCar("B",1,1,"N","FFF")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()

def test_Collided():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",0,0,"N","FFRFF")
    test_sim.test_AddCar("B",2,2,"S","LLL")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()

def test_MultipleCollided1():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",1,0,"N","F")
    test_sim.test_AddCar("B",1,1,"N","L")
    test_sim.test_AddCar("C",1,2,"S","F")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()

def test_MultipleCollided2():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",1,0,"N","F")
    test_sim.test_AddCar("B",1,1,"N","L")
    test_sim.test_AddCar("C",1,2,"S","F")
    test_sim.test_AddCar("D",9,9,"N","R")
    test_sim.test_AddCar("E",9,8,"N","F")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()

def test_Sample1():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",1,2,"N","FFRFFFFRRL")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()

def test_Sample2():
    test_sim = Session()
    test_sim.test_FieldConstruct(10,10)
    test_sim.test_AddCar("A",1,2,"N","FFRFFFFRRL")
    test_sim.test_AddCar("B",7,8,"W","FFLFFFFFFF")
    test_sim.DisplayCarDictList()
    test_sim.RunSimulation()


In [261]:
test_RotateLeft()

Your current list of cars are:
A , (0,0) N, LL
B , (1,1) W, LLLL
After Simulation the result is:
- A , (0,0) S
- B , (1,1) W


In [262]:
test_RotateRight()

Your current list of cars are:
A , (0,0) N, RRR
B , (1,1) E, RRRR
After Simulation the result is:
- A , (0,0) W
- B , (1,1) E


In [263]:
test_OutOfBound()

Your current list of cars are:
A , (0,0) N, F
After Simulation the result is:
- A , (0,0) N


In [231]:
test_NoCommand()

Your current list of cars are:
A , (0,0) N, 
After Simulation the result is:
- A , (0,0) N


In [219]:
test_NoCollided()

Your current list of cars are:
A , (0,0) N, FFF
B , (1,1) N, FFF
After Simulation the result is:
- A , (0,3) N
- B , (1,4) N


In [220]:
test_Collided()

Your current list of cars are:
A , (0,0) N, FFRFF
B , (2,2) S, LLL
After Simulation the result is:
- A , collides with B at (2,2) at step 5
- B , collides with A at (2,2) at step 5


In [221]:
test_MultipleCollided1()

Your current list of cars are:
A , (1,0) N, F
B , (1,1) N, L
C , (1,2) S, F
After Simulation the result is:
- A , collides with B,C at (1,1) at step 1
- B , collides with A,C at (1,1) at step 1
- C , collides with A,B at (1,1) at step 1


In [222]:
test_MultipleCollided2()

Your current list of cars are:
A , (1,0) N, F
B , (1,1) N, L
C , (1,2) S, F
D , (9,9) N, R
E , (9,8) N, F
After Simulation the result is:
- A , collides with B,C at (1,1) at step 1
- B , collides with A,C at (1,1) at step 1
- C , collides with A,B at (1,1) at step 1
- D , collides with E at (9,9) at step 1
- E , collides with D at (9,9) at step 1


In [223]:
test_Sample1()

Your current list of cars are:
A , (1,2) N, FFRFFFFRRL
After Simulation the result is:
- A , (5,4) S


In [224]:
test_Sample2()

Your current list of cars are:
A , (1,2) N, FFRFFFFRRL
B , (7,8) W, FFLFFFFFFF
After Simulation the result is:
- A , collides with B at (5,4) at step 7
- B , collides with A at (5,4) at step 7


In [264]:
#ipytest.run('-vv')

### 6. Main

In [166]:
exit = False
while exit == False:
    restart = False
    print("Welcome to Auto Driving Car Simulation!")
    sim = Session()
    sim.FieldConstruct()
    while restart == False:
        print("Please choose from the following options:\n[1] Add a car to field\n[2] Run simulation")
        try:
            options1 = int(input())
        except ValueError:
            print("Invalid input type.")
            continue
        if options1 == 1:
            sim.AddCar()
        elif options1 == 2:
            sim.RunSimulation()
            while True:
                print("Please choose from the following options:\n[1] Start over\n[2] Exit")
                try:
                    options2 = int(input())
                except ValueError:
                    print("Invalid input type.")
                    continue
                if options2 == 1:
                    restart = True
                    break
                elif options2 == 2:
                    restart = True
                    exit = True
                    break
                else:
                    print("Invalid input.")
        else:
            print("Invalid input.")
print("Thank you for running the simulation. Goodbye!")

Welcome to Auto Driving Car Simulation!
Please enter the width and height of the simulation field in x y format:


 10 10


you have created a field of 10 x 10 .
Please choose from the following options:
[1] Add a car to field
[2] Run simulation


 1


Please enter the name of the car:


...

Your current list of cars are:
A , (1,2) N, FFRFFFFRRL
B , (7,8) W, FFLFFFFFFF
Please choose from the following options:
[1] Add a car to field
[2] Run simulation


 2


After Simulation the result is:
- A , collides with B at (5,4) at step 7
- B , collides with A at (5,4) at step 7
Please choose from the following options:
[1] Start over
[2] Exit


 2


Thank you for running the simulation. Goodbye!
